## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-21-16-58-34 +0000,bbc,Trump says he will increase global tariffs to 15%,https://www.bbc.com/news/articles/cn8z48xwqn3o...
1,2026-02-21-16-52-05 +0000,nyt,Six Questions on the Future of Trump’s Tariffs,https://www.nytimes.com/2026/02/21/business/de...
2,2026-02-21-16-51-38 +0000,nyt,Trump Says He Will Raise Global Tariff to 15 P...,https://www.nytimes.com/2026/02/21/business/tr...
3,2026-02-21-16-47-43 +0000,nypost,Timothy Busfield pleads not guilty to child se...,https://nypost.com/2026/02/21/us-news/timothy-...
4,2026-02-21-16-46-07 +0000,wapo,Trump’s response to Supreme Court tariff block...,https://www.washingtonpost.com/business/2026/0...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2433/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,73
25,court,47
24,supreme,42
5,tariffs,34
10,tariff,29


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
161,2026-02-21-00-24-00 +0000,wsj,President Trump had just started a closed-door...,https://www.wsj.com/politics/policy/the-moment...,323
86,2026-02-21-10-30-00 +0000,wsj,The Supreme Court’s ruling that most of Trump’...,https://www.wsj.com/economy/trade/trump-lost-o...,256
170,2026-02-20-23-26-00 +0000,wsj,The Supreme Court decision to strike down Pres...,https://www.wsj.com/economy/trade/trump-tariff...,248
217,2026-02-20-20-56-39 +0000,wapo,"Trump imposes new tariffs, denounces Supreme C...",https://www.washingtonpost.com/national-securi...,247
12,2026-02-21-16-26-00 +0000,wsj,President Trump said he would increase to 15% ...,https://www.wsj.com/politics/policy/trump-incr...,241


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
161,323,2026-02-21-00-24-00 +0000,wsj,President Trump had just started a closed-door...,https://www.wsj.com/politics/policy/the-moment...
12,73,2026-02-21-16-26-00 +0000,wsj,President Trump said he would increase to 15% ...,https://www.wsj.com/politics/policy/trump-incr...
227,66,2026-02-20-20-30-00 +0000,wsj,"Dollar Down, Treasury Yields Slightly Up as Tr...",https://www.wsj.com/finance/currencies/asian-c...
284,34,2026-02-20-17-20-20 +0000,nypost,Iran covertly repositions strike drones amid R...,https://nypost.com/2026/02/20/world-news/iran-...
239,34,2026-02-20-19-56-29 +0000,nyt,Former Prince Andrew’s Arrest Upends Royal Fam...,https://www.nytimes.com/2026/02/20/world/europ...
13,31,2026-02-21-16-18-30 +0000,startribune,Live: U.S. women’s curling team loses to Canad...,https://www.startribune.com/2026-winter-olympi...
40,30,2026-02-21-13-53-38 +0000,nypost,Shocking video shows group of NYC teens climbi...,https://nypost.com/2026/02/21/us-news/shock-vi...
37,28,2026-02-21-13-58-07 +0000,nypost,How savvy GOPers plan to grill Bill and Hillar...,https://nypost.com/2026/02/21/us-news/in-the-h...
30,27,2026-02-21-14-21-18 +0000,nypost,DOJ axed NY judge who handed out more asylum c...,https://nypost.com/2026/02/21/us-news/youre-fi...
175,26,2026-02-20-23-09-11 +0000,nypost,Feds won’t charge Labor Secretary Lori Chavez-...,https://nypost.com/2026/02/20/us-news/feds-won...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
